In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchmetrics.regression import R2Score
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import random
import pandas as pd

# 기기 설정 (GPU 또는 CPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 사용자 정의 Dataset 클래스: feature와 target을 텐서로 변환
class MyDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.FloatTensor(features)  # 피처 텐서화
        self.targets = torch.FloatTensor(targets).unsqueeze(1)  # 타겟 텐서화 (회귀이므로 2차원)
    
    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

# 모델 클래스
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self, in_features, out_target, hidden_layers, output_activation=None):
        """
        in_features: 입력 피처 수
        out_target: 출력 노드 수 (회귀=1, 이진분류=1, 다중분류=클래스 수)
        hidden_layers: 은닉층 노드 수를 리스트로 받아서 은닉층의 개수와 각 층의 노드 수를 설정
        output_activation: 출력층에 사용할 활성화 함수 (None, 'sigmoid', 'softmax' 중 하나 선택)
        """
        super(MyModel, self).__init__()
        
        self.input_layer = nn.Linear(in_features, hidden_layers[0])
        
        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_layers) - 1):
            self.hidden_layers.append(nn.Linear(hidden_layers[i], hidden_layers[i+1]))
        
        self.output_layer = nn.Linear(hidden_layers[-1], out_target)
        
        # 출력층 활성화 함수 설정 (None, 'sigmoid', 'softmax')
        self.output_activation = output_activation

    def forward(self, X):
        y = self.input_layer(X)
        y = F.relu(y)
        
        for layer in self.hidden_layers:
            y = layer(y)
            y = F.relu(y)
        
        y = self.output_layer(y)
        
        if self.output_activation == 'sigmoid':
            y = torch.sigmoid(y)
        elif self.output_activation == 'softmax':
            y = F.softmax(y, dim=1) 
        
        return y


# 학습용 함수
def epochTraining(model, train_loader, lossfunc, optimizer):
    model.train()  # 모델 학습 모드 전환
    loss_total = 0
    score_total = 0
    r2_metric = R2Score().to(DEVICE)  # R2Score는 회귀용
    for X_train, y_train in train_loader:
        X_train, y_train = X_train.to(DEVICE), y_train.to(DEVICE)
        pre_y = model(X_train)
        loss = lossfunc(pre_y, y_train)
        loss_total += loss.item()
        score_total += r2_metric(pre_y, y_train).item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss_total / len(train_loader), score_total / len(train_loader)

# 검증용 함수
def testing(model, val_loader, lossfunc):
    model.eval()
    loss_total = 0
    score_total = 0
    r2_metric = R2Score().to(DEVICE)  # R2Score는 회귀용
    with torch.no_grad():
        for X_val, y_val in val_loader:
            X_val, y_val = X_val.to(DEVICE), y_val.to(DEVICE)
            pre_y = model(X_val)
            loss = lossfunc(pre_y, y_val)
            loss_total += loss.item()
            score_total += r2_metric(pre_y, y_val).item()
    return loss_total / len(val_loader), score_total / len(val_loader)

# 학습 실행 함수
def run_training(model, train_features, train_targets, val_features, val_targets, epochs=10, batch_size=32):
    # 학습 데이터셋과 검증 데이터셋 텐서화
    train_dataset = MyDataset(train_features, train_targets)
    val_dataset = MyDataset(val_features, val_targets)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    lossfunc = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    TV_LOSS = {'Train': [], 'Val': []}
    TV_SCORE = {'Train': [], 'Val': []}
    
    for epoch in range(epochs):
        train_loss, train_score = epochTraining(model, train_loader, lossfunc, optimizer)
        val_loss, val_score = testing(model, val_loader, lossfunc)
        
        TV_LOSS['Train'].append(train_loss)
        TV_SCORE['Train'].append(train_score)
        TV_LOSS['Val'].append(val_loss)
        TV_SCORE['Val'].append(val_score)
        
        print(f'Epoch [{epoch+1}/{epochs}]')
        print(f'- [TRAIN] Loss: {train_loss:.4f}, R2: {train_score:.4f}')
        print(f'- [VALID] Loss: {val_loss:.4f}, R2: {val_score:.4f}')
    
    return TV_LOSS, TV_SCORE


In [8]:
# 데이터 로드 및 전처리
iris = load_iris()
features = pd.DataFrame(iris.data, columns=iris.feature_names)

# 예측할 타겟 (sepal width)와 나머지 피처 분리
target = features['sepal width (cm)'].values
features = features.drop(columns=['sepal width (cm)']).values

# MinMaxScaler 적용
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# 데이터를 train, val, test로 분할 (train 60%, val 20%, test 20%)
X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# 모델 정의 (입력 피처 3개, 은닉층 64, 128, 64, 출력 1개)
model = MyModel(in_features=3, out_target=1, hidden_layers=[64, 128, 64],output_activation=None).to(DEVICE)

# 학습 실행
TV_LOSS, TV_SCORE = run_training(model, X_train, y_train, X_val, y_val, epochs=1000, batch_size=16)

Epoch [1/1000]
- [TRAIN] Loss: 8.5114, R2: -46.5082
- [VALID] Loss: 6.6238, R2: -52.1435
Epoch [2/1000]
- [TRAIN] Loss: 6.9475, R2: -38.1884
- [VALID] Loss: 5.1751, R2: -40.6104
Epoch [3/1000]
- [TRAIN] Loss: 5.1439, R2: -30.3751
- [VALID] Loss: 3.3798, R2: -26.2958
Epoch [4/1000]
- [TRAIN] Loss: 3.0384, R2: -16.2672
- [VALID] Loss: 1.5378, R2: -11.5467
Epoch [5/1000]
- [TRAIN] Loss: 1.2647, R2: -7.3958
- [VALID] Loss: 0.5945, R2: -3.8231
Epoch [6/1000]
- [TRAIN] Loss: 0.9089, R2: -4.0516
- [VALID] Loss: 0.8031, R2: -5.2740
Epoch [7/1000]
- [TRAIN] Loss: 0.9759, R2: -4.6491
- [VALID] Loss: 0.5760, R2: -3.5269
Epoch [8/1000]
- [TRAIN] Loss: 0.6550, R2: -2.3853
- [VALID] Loss: 0.3886, R2: -2.1267
Epoch [9/1000]
- [TRAIN] Loss: 0.5325, R2: -1.7806
- [VALID] Loss: 0.3491, R2: -1.8411
Epoch [10/1000]
- [TRAIN] Loss: 0.4731, R2: -1.9550
- [VALID] Loss: 0.2778, R2: -1.2470
Epoch [11/1000]
- [TRAIN] Loss: 0.3845, R2: -1.0066
- [VALID] Loss: 0.2331, R2: -0.8431
Epoch [12/1000]
- [TRAIN] Loss: 0

In [9]:
# 테스트 성능 평가
def evaluate_on_test(model, X_test, y_test):
    model.eval()
    test_dataset = MyDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    r2_metric = R2Score().to(DEVICE)
    lossfunc = nn.MSELoss()
    
    with torch.no_grad():
        total_r2 = 0
        total_loss = 0
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            y_pred = model(X_batch)
            total_r2 += r2_metric(y_pred, y_batch).item()
            total_loss += lossfunc(y_pred, y_batch).item()
    
    avg_r2 = total_r2 / len(test_loader)
    avg_loss = total_loss / len(test_loader)
    
    print(f'[TEST] Loss: {avg_loss:.4f}, R2 Score: {avg_r2:.4f}')
    
    # 랜덤으로 10개의 예측값과 실제값을 비교
    random_indices = random.sample(range(len(X_test)), 10)
    X_random = torch.FloatTensor(X_test[random_indices]).to(DEVICE)
    y_random = y_test[random_indices]
    
    with torch.no_grad():
        y_pred_random = model(X_random).cpu().numpy().flatten()
    
    print("\nRandom 10 samples comparison:")
    print(f"{'Index':<5} {'Predicted':<10} {'Actual':<10}")
    for i, (pred, actual) in enumerate(zip(y_pred_random, y_random)):
        print(f"{i:<5} {pred:<10.4f} {actual:<10.4f}")


In [11]:
evaluate_on_test(model, X_test, y_test)

[TEST] Loss: 0.1041, R2 Score: 0.2914

Random 10 samples comparison:
Index Predicted  Actual    
0     3.4480     3.4000    
1     3.3163     3.1000    
2     3.5113     3.8000    
3     3.2015     3.0000    
4     3.2232     3.2000    
5     2.8925     3.0000    
6     2.5868     2.5000    
7     3.2145     3.0000    
8     3.4579     2.6000    
9     3.2738     3.2000    
